In [ ]:
import requests
import pandas as pd
import time

"""Petfinder API to pull listings for adoptable pets with pagination and retry logic."""


TOKEN_URL = "https://api.petfinder.com/v2/oauth2/token"


# 🔐 Step 1: Authenticate and obtain API access token
auth_data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}
response = requests.post(TOKEN_URL, data=auth_data)
access_token = response.json().get("access_token")

if not access_token:
    print(f"❌ Failed to obtain access token: {response.json()}")
    exit()

# 🏗 Step 2: Define headers for authentication
headers = {"Authorization": f"Bearer {access_token}"}

# 🐶 Step 3: Fetch adoptable pets using pagination with retry logic
PET_URL = "https://api.petfinder.com/v2/animals"
all_pets = []  # Store all pets data

page = 1  # Start from first page
while True:
    params = {"type": "dog", "location": "75001", "limit": 100, "page": page}
    success = False
    attempts = 0
    while not success and attempts < 3:
        response = requests.get(PET_URL, params=params, headers=headers)
        if response.status_code == 200:
            pet_data = response.json()["animals"]
            success = True  # Successful retrieval
        else:
            print(f"❌ Failed to fetch pets on page {page}: {response.status_code} - attempt {attempts+1}")
            attempts += 1
            time.sleep(5)  # Wait 5 seconds before retrying

    if not success:
        print(f"❌ Giving up on page {page}.")
        break

    if not pet_data:  # If no more pets are returned, stop pagination
        break

    all_pets.extend(pet_data)  # Append current batch of pets
    page += 1  # Move to next page

# 📊 Convert all pet data to a DataFrame
df_pets = pd.json_normalize(all_pets)
# 🎯 Select relevant columns
#df_pet = df_pets[["id", "name", "breeds.primary", "age", "organization_id", "url", "contact.address.city", "contact.address.state"]]
#df_pet.columns = ["Pet ID", "Name", "Breed", "Age", "Shelter ID", "Adoption Link", "City", "State"]

# 🏠 Step 4: Fetch shelter details dynamically
shelter_ids = df_pets["Shelter ID"].dropna().unique()
df_shelters = pd.DataFrame()

for shelter_id in shelter_ids:
    ORG_URL = f"https://api.petfinder.com/v2/organizations/{shelter_id}"
    response = requests.get(ORG_URL, headers=headers)
    if response.status_code == 200:
        shelter_data = response.json()["organization"]
        df_temp = pd.json_normalize(shelter_data)
        df_shelters = pd.concat([df_shelters, df_temp], ignore_index=True)

# 🎯 Select relevant columns for shelters
#df_shelters = df_shelters[["id", "name", "address.city", "address.state", "phone", "url"]]
#df_shelters.columns = ["Shelter ID", "Shelter Name", "City", "State", "Phone", "Website"]

# 🔗 Merge Pet & Shelter DataFrames
df_combined = df_pets.merge(df_shelters, on="Shelter ID", how="left")

# 💾 Step 5: Save DataFrames to Excel and CSV
df_pets.to_csv("adoptable_pets.csv", index=False)
df_shelters.to_csv("shelter_details.csv", index=False)
df_combined.to_csv("adoptable_pets_shelters.csv", index=False)
#df_pets.to_csv("original_adoptable_pets.csv", index=False)
df_combined.to_excel("adoptable_pets_shelters.xlsx", index=False, sheet_name="Adoption Data")

print("✅ Data successfully saved! 🎉")
print("📂 CSV files: 'adoptable_pets.csv', 'shelter_details.csv', 'adoptable_pets_shelters.csv',original_adoptable_pets.csv")
print("📂 Excel file: 'adoptable_pets_shelters.xlsx'")
print(df_combined.head())